In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [3]:
#Loading Data
data = pd.read_csv('C:/Pythondatasets/Dataset_Sarajevo_Sa_Diskretnim_Vremenom.csv', parse_dates=True, index_col='Date (LT)')
data

,Site,Parameter,Year,Month,Day,Hour,NowCast Conc.,AQI,AQI Category,Raw Conc.,Conc. Unit,Duration,QC Name,pomocna,diskretno_vrijeme
Date (LT),,,,,,,,,,,,,,,
2018-05-22 17:00:00,Sarajevo,PM2.5 - Principal,2018,5,22,17,10.0,42,Good,7.0,UG/M3,1 Hr,Valid,NaN,0
2018-05-22 18:00:00,Sarajevo,PM2.5 - Principal,2018,5,22,18,15.7,59,Moderate,20.0,UG/M3,1 Hr,Valid,5/22/2018 17:00,1
2018-05-22 19:00:00,Sarajevo,PM2.5 - Principal,2018,5,22,19,14.2,55,Moderate,13.0,UG/M3,1 Hr,Valid,5/22/2018 18:00,2
2018-05-22 20:00:00,Sarajevo,PM2.5 - Principal,2018,5,22,20,16.7,61,Moderate,19.0,UG/M3,1 Hr,Valid,5/22/2018 19:00,3
2018-05-22 21:00:00,Sarajevo,PM2.5 - Principal,2018,5,22,21,13.8,55,Moderate,11.0,UG/M3,1 Hr,Valid,5/22/2018 20:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 21:00:00,Sarajevo,PM2.5 - Principal,2023,4,30,21,15.2,58,Moderate,21.0,UG/M3,1 Hr,Valid,4/30/2023 20:00,41472
2023-04-30 22:00:00,Sarajevo,PM2.5 - Principal,2023,4,30,22,23.6,75,Moderate,32.0,UG/M3,1 Hr,Valid,4/30/2023 21:00,41473
2023-04-30 23:00:00,Sarajevo,PM2.5 - Principal,2023,4,30,23,23.8,76,Moderate,24.0,UG/M3,1 Hr,Valid,4/30/2023 22:00,41474


In [4]:
series = data[['AQI']]
series

,AQI
Date (LT),
2018-05-22 17:00:00,42
2018-05-22 18:00:00,59
2018-05-22 19:00:00,55
2018-05-22 20:00:00,61
2018-05-22 21:00:00,55
...,...
2023-04-30 21:00:00,58
2023-04-30 22:00:00,75
2023-04-30 23:00:00,76


In [5]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from pandas import datetime

C:\Users\Korisnik\AppData\Local\Temp\ipykernel_10372\734045020.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [6]:
# one-step sarima forecast
def sarima_forecast(history, config):
 order, sorder, trend = config
 # define model
 model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
 # fit model
 model_fit = model.fit(disp=False)
 # make one step forecast
 yhat = model_fit.predict(len(history), len(history))
 return yhat[0]

In [7]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
 return sqrt(mean_squared_error(actual, predicted))

In [8]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
 return data[:-n_test], data[-n_test:]

In [9]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    print("Walk forward begin")
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    print("error in walk forward ", error)
    return error

In [10]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    print("Score model")
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        print("Prvi if")
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                ilterwarnings("ignore")
                print("Prije resulta")
                result = walk_forward_validation(data, n_test, cfg)
                print("Result: ", result)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

In [11]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    print("Grid search")
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
        print('dje je zapelo')

    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

In [12]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    print("Sarima configs")
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n','c','t','ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models



In [ ]:
data = series.values
print(data)
# data split
n_test = 12
print("heeej")
# model configs
cfg_list = sarima_configs(seasonal=[0, 12])
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

[[42]
 [59]
 [55]
 ...
 [76]
 [79]
 [79]]
heeej
Sarima configs
Grid search
